<a href="https://colab.research.google.com/github/tanvir-talha058/Machine-Learning/blob/main/Regional_Dialect.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas openpyxl transformers datasets sentencepiece


In [ ]:
import pandas as pd

# Load your Excel file
df = pd.read_excel('/content/dataset.xlsx')

# Show sample rows
df.head()


,Promito,Rajshahi,Sylhet,Chottogram,Rangpur
0,তুমি কি করছো?,তুমি কী কইরছো,তুমি কিতা করো?,তুঁই কিরর ?,কি কইরবান নাকছেন তোমরা???
1,তুমি কোথা থেকে আসছো?,কোতি থেকে অ্যাসছো?,তুমি কইথাকি আইছ?,তুঁই হত্তুন আইয়্যির ?,কোনটে থাকি আসচেন বাহে তোমরা?
2,আপনি কোথায় যাচ্ছেন?,কতি জ্যাছেন?,আফনে কই যাইরাইন?,অনে হঁডে যর?,কোনটে জান বাহে তোমরা?
3,আপনার দিনকাল কেমন যাচ্ছে?,দিন ক্যামন জ্যাছে আপনার?,কিরম যায় বা তোমার দিন?,অঁনর দিনহাল ক্যান চলের ওয়া??,বাহে আপনার দিনকাল ক্যামনে যাচ্ছে?
4,আপনার সপ্তাহান্তের পরিকল্পনা কি?,সপ্তাহ শ্যাসে কি কইরবেন,আফনার সামনের সাপ্তাত পরিকল্পনা কিতা?,"ত, শুক্কু-শনিবারে কি গরিবে চিন্তে গইজ্জু??",বাহে আপনার সপ্তাহের পরিকল্পনা কি?


In [ ]:
pairs = []

for col in df.columns[1:]:  # Skip 'Promito'
    for i in range(len(df)):
        source = df[col][i]
        target = df['Promito'][i]
        if pd.notna(source) and pd.notna(target):
            pairs.append({'input': source, 'target': target})

df_pairs = pd.DataFrame(pairs)
df_pairs.to_csv("dialect_translation.csv", index=False)
df_pairs.head()


,input,target
0,তুমি কী কইরছো,তুমি কি করছো?
1,কোতি থেকে অ্যাসছো?,তুমি কোথা থেকে আসছো?
2,কতি জ্যাছেন?,আপনি কোথায় যাচ্ছেন?
3,দিন ক্যামন জ্যাছে আপনার?,আপনার দিনকাল কেমন যাচ্ছে?
4,সপ্তাহ শ্যাসে কি কইরবেন,আপনার সপ্তাহান্তের পরিকল্পনা কি?


In [ ]:
from datasets import Dataset

# Load into Hugging Face Dataset
dataset = Dataset.from_pandas(df_pairs)

# Train/Test Split (90% train, 10% test)
dataset = dataset.train_test_split(test_size=0.1)
dataset


DatasetDict({
    train: Dataset({
        features: ['input', 'target'],
        num_rows: 1270
    })
    test: Dataset({
        features: ['input', 'target'],
        num_rows: 142
    })
})

In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

model_name = "google/mt5-small"
tokenizer = T5Tokenizer.from_pretrained(model_name)
model = T5ForConditionalGeneration.from_pretrained(model_name)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used s

In [ ]:
def preprocess(example):
    input_text = "translate Bangla: " + example["input"]
    model_input = tokenizer(input_text, truncation=True, padding="max_length", max_length=128)
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(example["target"], truncation=True, padding="max_length", max_length=128)
    model_input["labels"] = labels["input_ids"]
    return model_input

tokenized_dataset = dataset.map(preprocess)


Map:   0%|          | 0/1270 [00:00<?, ? examples/s]

/usr/local/lib/python3.11/dist-packages/transformers/tokenization_utils_base.py:3935: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/142 [00:00<?, ? examples/s]

In [ ]:

!pip install transformers==4.40.0  # Or latest stable version


In [ ]:
import transformers
print(transformers.__version__)


4.39.3


In [ ]:
from transformers import TrainingArguments, Trainer


RuntimeError: Failed to import transformers.trainer because of the following error (look up to see its traceback):
cannot import name 'EncoderDecoderCache' from 'transformers' (/usr/local/lib/python3.11/dist-packages/transformers/__init__.py)